# **About this notebook**
WiDS Datathon 2023
Adapting to Climate Change by Improving Extreme Weather Forecasts.

We are using a pre-prepared dataset consisting of weather and climate information for a number of US locations, for a number of start dates for the two-week observation, as well as the forecasted temperature and precipitation from a number of weather forecast models (we will reveal the source of our dataset after the competition closes). Each row in the data corresponds to a single location and a single start date for the two-week period.
Data:
You are provided with two datasets:
train_data.csv: the training dataset, where contest-tmp2m-14d__tmp2m, the arithmetic mean of the max and min observed temperature over the next 14 days for each location and start date, is provided.
test_data.csv: the test dataset, where we withhold the true value of contest-tmp2m-14d__tmp2m for each row.

# **Objective**

Your task is to predict the arithmetic mean of the maximum and minimum temperature over the next 14 days, for each location and start date.


# Import Libraries 

In [1]:
import numpy as np 
import pandas as pd 
from datetime import datetime

# Load the data 

In [2]:
train_df = pd.read_csv('/kaggle/input/widsdatathon2023/train_data.csv')
display(train_df.head().style.set_caption('Train data'))

test_df = pd.read_csv('/kaggle/input/widsdatathon2023/test_data.csv')
display(test_df.head().style.set_caption('Test data'))

,index,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,nmme0-tmp2m-34w__gfdlflora0,nmme0-tmp2m-34w__gfdlflorb0,nmme0-tmp2m-34w__gfdl0,nmme0-tmp2m-34w__nasa0,nmme0-tmp2m-34w__nmme0mean,contest-wind-h10-14d__wind-hgt-10,nmme-tmp2m-56w__cancm3,nmme-tmp2m-56w__cancm4,nmme-tmp2m-56w__ccsm3,nmme-tmp2m-56w__ccsm4,nmme-tmp2m-56w__cfsv2,nmme-tmp2m-56w__gfdl,nmme-tmp2m-56w__gfdlflora,nmme-tmp2m-56w__gfdlflorb,nmme-tmp2m-56w__nasa,nmme-tmp2m-56w__nmmemean,contest-rhum-sig995-14d__rhum,nmme-prate-34w__cancm3,nmme-prate-34w__cancm4,nmme-prate-34w__ccsm3,nmme-prate-34w__ccsm4,nmme-prate-34w__cfsv2,nmme-prate-34w__gfdl,nmme-prate-34w__gfdlflora,nmme-prate-34w__gfdlflorb,nmme-prate-34w__nasa,nmme-prate-34w__nmmemean,contest-wind-h100-14d__wind-hgt-100,nmme0-prate-56w__cancm30,nmme0-prate-56w__cancm40,nmme0-prate-56w__ccsm30,nmme0-prate-56w__ccsm40,nmme0-prate-56w__cfsv20,nmme0-prate-56w__gfdlflora0,nmme0-prate-56w__gfdlflorb0,nmme0-prate-56w__gfdl0,nmme0-prate-56w__nasa0,nmme0-prate-56w__nmme0mean,nmme0-prate-34w__cancm30,nmme0-prate-34w__cancm40,nmme0-prate-34w__ccsm30,nmme0-prate-34w__ccsm40,nmme0-prate-34w__cfsv20,nmme0-prate-34w__gfdlflora0,nmme0-prate-34w__gfdlflorb0,nmme0-prate-34w__gfdl0,nmme0-prate-34w__nasa0,nmme0-prate-34w__nmme0mean,contest-tmp2m-14d__tmp2m,contest-slp-14d__slp,contest-wind-vwnd-925-14d__wind-vwnd-925,nmme-prate-56w__cancm3,nmme-prate-56w__cancm4,nmme-prate-56w__ccsm3,nmme-prate-56w__ccsm4,nmme-prate-56w__cfsv2,nmme-prate-56w__gfdl,nmme-prate-56w__gfdlflora,nmme-prate-56w__gfdlflorb,nmme-prate-56w__nasa,nmme-prate-56w__nmmemean,contest-pres-sfc-gauss-14d__pres,contest-wind-uwnd-250-14d__wind-uwnd-250,nmme-tmp2m-34w__cancm3,nmme-tmp2m-34w__cancm4,nmme-tmp2m-34w__ccsm3,nmme-tmp2m-34w__ccsm4,nmme-tmp2m-34w__cfsv2,nmme-tmp2m-34w__gfdl,nmme-tmp2m-34w__gfdlflora,nmme-tmp2m-34w__gfdlflorb,nmme-tmp2m-34w__nasa,nmme-tmp2m-34w__nmmemean,contest-prwtr-eatm-14d__prwtr,contest-wind-vwnd-250-14d__wind-vwnd-250,contest-precip-14d__precip,contest-wind-h850-14d__wind-hgt-850,contest-wind-uwnd-925-14d__wind-uwnd-925,contest-wind-h500-14d__wind-hgt-500,cancm30,cancm40,ccsm30,ccsm40,cfsv20,gfdlflora0,gfdlflorb0,gfdl0,nasa0,nmme0mean,climateregions__climateregion,elevation__elevation,wind-vwnd-250-2010-1,wind-vwnd-250-2010-2,wind-vwnd-250-2010-3,wind-vwnd-250-2010-4,wind-vwnd-250-2010-5,wind-vwnd-250-2010-6,wind-vwnd-250-2010-7,wind-vwnd-250-2010-8,wind-vwnd-250-2010-9,wind-vwnd-250-2010-10,wind-vwnd-250-2010-11,wind-vwnd-250-2010-12,wind-vwnd-250-2010-13,wind-vwnd-250-2010-14,wind-vwnd-250-2010-15,wind-vwnd-250-2010-16,wind-vwnd-250-2010-17,wind-vwnd-250-2010-18,wind-vwnd-250-2010-19,wind-vwnd-250-2010-20,wind-uwnd-250-2010-1,wind-uwnd-250-2010-2,wind-uwnd-250-2010-3,wind-uwnd-250-2010-4,wind-uwnd-250-2010-5,wind-uwnd-250-2010-6,wind-uwnd-250-2010-7,wind-uwnd-250-2010-8,wind-uwnd-250-2010-9,wind-uwnd-250-2010-10,wind-uwnd-250-2010-11,wind-uwnd-250-2010-12,wind-uwnd-250-2010-13,wind-uwnd-250-2010-14,wind-uwnd-250-2010-15,wind-uwnd-250-2010-16,wind-uwnd-250-2010-17,wind-uwnd-250-2010-18,wind-uwnd-250-2010-19,wind-uwnd-250-2010-20,mjo1d__phase,mjo1d__amplitude,mei__mei,mei__meirank,mei__nip,wind-hgt-850-2010-1,wind-hgt-850-2010-2,wind-hgt-850-2010-3,wind-hgt-850-2010-4,wind-hgt-850-2010-5,wind-hgt-850-2010-6,wind-hgt-850-2010-7,wind-hgt-850-2010-8,wind-hgt-850-2010-9,wind-hgt-850-2010-10,sst-2010-1,sst-2010-2,sst-2010-3,sst-2010-4,sst-2010-5,sst-2010-6,sst-2010-7,sst-2010-8,sst-2010-9,sst-2010-10,wind-hgt-500-2010-1,wind-hgt-500-2010-2,wind-hgt-500-2010-3,wind-hgt-500-2010-4,wind-hgt-500-2010-5,wind-hgt-500-2010-6,wind-hgt-500-2010-7,wind-hgt-500-2010-8,wind-hgt-500-2010-9,wind-hgt-500-2010-10,icec-2010-1,icec-2010-2,icec-2010-3,icec-2010-4,icec-2010-5,icec-2010-6,icec-2010-7,icec-2010-8,icec-2010-9,icec-2010-10,wind-uwnd-925-2010-1,wind-uwnd-925-2010-2,wind-uwnd-925-2010-3,wind-uwnd-925-2010-4,wind-uwnd-925-2010

,index,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,nmme0-tmp2m-34w__gfdlflora0,nmme0-tmp2m-34w__gfdlflorb0,nmme0-tmp2m-34w__gfdl0,nmme0-tmp2m-34w__nasa0,nmme0-tmp2m-34w__nmme0mean,contest-wind-h10-14d__wind-hgt-10,nmme-tmp2m-56w__cancm3,nmme-tmp2m-56w__cancm4,nmme-tmp2m-56w__ccsm3,nmme-tmp2m-56w__ccsm4,nmme-tmp2m-56w__cfsv2,nmme-tmp2m-56w__gfdl,nmme-tmp2m-56w__gfdlflora,nmme-tmp2m-56w__gfdlflorb,nmme-tmp2m-56w__nasa,nmme-tmp2m-56w__nmmemean,contest-rhum-sig995-14d__rhum,nmme-prate-34w__cancm3,nmme-prate-34w__cancm4,nmme-prate-34w__ccsm3,nmme-prate-34w__ccsm4,nmme-prate-34w__cfsv2,nmme-prate-34w__gfdl,nmme-prate-34w__gfdlflora,nmme-prate-34w__gfdlflorb,nmme-prate-34w__nasa,nmme-prate-34w__nmmemean,contest-wind-h100-14d__wind-hgt-100,nmme0-prate-56w__cancm30,nmme0-prate-56w__cancm40,nmme0-prate-56w__ccsm30,nmme0-prate-56w__ccsm40,nmme0-prate-56w__cfsv20,nmme0-prate-56w__gfdlflora0,nmme0-prate-56w__gfdlflorb0,nmme0-prate-56w__gfdl0,nmme0-prate-56w__nasa0,nmme0-prate-56w__nmme0mean,nmme0-prate-34w__cancm30,nmme0-prate-34w__cancm40,nmme0-prate-34w__ccsm30,nmme0-prate-34w__ccsm40,nmme0-prate-34w__cfsv20,nmme0-prate-34w__gfdlflora0,nmme0-prate-34w__gfdlflorb0,nmme0-prate-34w__gfdl0,nmme0-prate-34w__nasa0,nmme0-prate-34w__nmme0mean,contest-slp-14d__slp,contest-wind-vwnd-925-14d__wind-vwnd-925,nmme-prate-56w__cancm3,nmme-prate-56w__cancm4,nmme-prate-56w__ccsm3,nmme-prate-56w__ccsm4,nmme-prate-56w__cfsv2,nmme-prate-56w__gfdl,nmme-prate-56w__gfdlflora,nmme-prate-56w__gfdlflorb,nmme-prate-56w__nasa,nmme-prate-56w__nmmemean,contest-pres-sfc-gauss-14d__pres,contest-wind-uwnd-250-14d__wind-uwnd-250,nmme-tmp2m-34w__cancm3,nmme-tmp2m-34w__cancm4,nmme-tmp2m-34w__ccsm3,nmme-tmp2m-34w__ccsm4,nmme-tmp2m-34w__cfsv2,nmme-tmp2m-34w__gfdl,nmme-tmp2m-34w__gfdlflora,nmme-tmp2m-34w__gfdlflorb,nmme-tmp2m-34w__nasa,nmme-tmp2m-34w__nmmemean,contest-prwtr-eatm-14d__prwtr,contest-wind-vwnd-250-14d__wind-vwnd-250,contest-precip-14d__precip,contest-wind-h850-14d__wind-hgt-850,contest-wind-uwnd-925-14d__wind-uwnd-925,contest-wind-h500-14d__wind-hgt-500,cancm30,cancm40,ccsm30,ccsm40,cfsv20,gfdlflora0,gfdlflorb0,gfdl0,nasa0,nmme0mean,climateregions__climateregion,elevation__elevation,wind-vwnd-250-2010-1,wind-vwnd-250-2010-2,wind-vwnd-250-2010-3,wind-vwnd-250-2010-4,wind-vwnd-250-2010-5,wind-vwnd-250-2010-6,wind-vwnd-250-2010-7,wind-vwnd-250-2010-8,wind-vwnd-250-2010-9,wind-vwnd-250-2010-10,wind-vwnd-250-2010-11,wind-vwnd-250-2010-12,wind-vwnd-250-2010-13,wind-vwnd-250-2010-14,wind-vwnd-250-2010-15,wind-vwnd-250-2010-16,wind-vwnd-250-2010-17,wind-vwnd-250-2010-18,wind-vwnd-250-2010-19,wind-vwnd-250-2010-20,wind-uwnd-250-2010-1,wind-uwnd-250-2010-2,wind-uwnd-250-2010-3,wind-uwnd-250-2010-4,wind-uwnd-250-2010-5,wind-uwnd-250-2010-6,wind-uwnd-250-2010-7,wind-uwnd-250-2010-8,wind-uwnd-250-2010-9,wind-uwnd-250-2010-10,wind-uwnd-250-2010-11,wind-uwnd-250-2010-12,wind-uwnd-250-2010-13,wind-uwnd-250-2010-14,wind-uwnd-250-2010-15,wind-uwnd-250-2010-16,wind-uwnd-250-2010-17,wind-uwnd-250-2010-18,wind-uwnd-250-2010-19,wind-uwnd-250-2010-20,mjo1d__phase,mjo1d__amplitude,mei__mei,mei__meirank,mei__nip,wind-hgt-850-2010-1,wind-hgt-850-2010-2,wind-hgt-850-2010-3,wind-hgt-850-2010-4,wind-hgt-850-2010-5,wind-hgt-850-2010-6,wind-hgt-850-2010-7,wind-hgt-850-2010-8,wind-hgt-850-2010-9,wind-hgt-850-2010-10,sst-2010-1,sst-2010-2,sst-2010-3,sst-2010-4,sst-2010-5,sst-2010-6,sst-2010-7,sst-2010-8,sst-2010-9,sst-2010-10,wind-hgt-500-2010-1,wind-hgt-500-2010-2,wind-hgt-500-2010-3,wind-hgt-500-2010-4,wind-hgt-500-2010-5,wind-hgt-500-2010-6,wind-hgt-500-2010-7,wind-hgt-500-2010-8,wind-hgt-500-2010-9,wind-hgt-500-2010-10,icec-2010-1,icec-2010-2,icec-2010-3,icec-2010-4,icec-2010-5,icec-2010-6,icec-2010-7,icec-2010-8,icec-2010-9,icec-2010-10,wind-uwnd-925-2010-1,wind-uwnd-925-2010-2,wind-uwnd-925-2010-3,wind-uwnd-925-2010-4,wind-uwnd-925-2010-5,wind-uwnd-925-2010-6,w

# Exploratory data analysis 

To have a first glance of the data, we will do some exploratory data analysis (EDA). 

First, we noticed that the test_df is around 8.3% of the size of the train_df.

Second, the dfs contain eight columns with NaN's. 


In [3]:
train_df.shape 

(375734, 246)

In [4]:
test_df.shape

(31354, 245)

In [5]:
train_df.isna().sum()

index                                 0
lat                                   0
lon                                   0
startdate                             0
contest-pevpr-sfc-gauss-14d__pevpr    0
                                     ..
wind-vwnd-925-2010-16                 0
wind-vwnd-925-2010-17                 0
wind-vwnd-925-2010-18                 0
wind-vwnd-925-2010-19                 0
wind-vwnd-925-2010-20                 0
Length: 246, dtype: int64

We will want to be sure if there is any column that has NAN in our train_data above.

In [6]:
train_df.columns[train_df.isna().any()].tolist()

['nmme0-tmp2m-34w__ccsm30',
 'nmme-tmp2m-56w__ccsm3',
 'nmme-prate-34w__ccsm3',
 'nmme0-prate-56w__ccsm30',
 'nmme0-prate-34w__ccsm30',
 'nmme-prate-56w__ccsm3',
 'nmme-tmp2m-34w__ccsm3',
 'ccsm30']

In [7]:
# making a smaller copy of the data (one with less columns with NANs excluding startdate and longitude) 
train_df_form=train_df[['startdate','lon','nmme0-tmp2m-34w__ccsm30', 'nmme-tmp2m-56w__ccsm3', 'nmme-prate-34w__ccsm3', 'nmme0-prate-56w__ccsm30', 'nmme0-prate-34w__ccsm30', 'nmme-prate-56w__ccsm3', 'nmme-tmp2m-34w__ccsm3', 'ccsm30']].copy()
train_df_form.tail(50)

,startdate,lon,nmme0-tmp2m-34w__ccsm30,nmme-tmp2m-56w__ccsm3,nmme-prate-34w__ccsm3,nmme0-prate-56w__ccsm30,nmme0-prate-34w__ccsm30,nmme-prate-56w__ccsm3,nmme-tmp2m-34w__ccsm3,ccsm30
375684,7/13/16,0.866667,18.72,22.36,20.04,21.86,21.86,20.04,22.36,18.72
375685,7/14/16,0.866667,18.72,22.36,20.04,21.86,21.86,20.04,22.36,18.72
375686,7/15/16,0.866667,18.72,22.36,20.04,21.86,21.86,20.04,22.36,18.72
375687,7/16/16,0.866667,18.72,22.36,20.04,21.86,21.86,20.04,22.36,18.72
375688,7/17/16,0.866667,18.72,22.36,20.04,21.86,21.86,20.04,22.36,18.72
375689,7/18/16,0.866667,18.72,22.36,20.04,21.86,21.86,20.04,22.36,18.72
375690,7/19/16,0.866667,18.72,22.36,19.23,21.86,21.86,20.04,22.24,18.72
375691,7/20/16,0.866667,18.72,22.24,18.42,21.86,21.86,19.23,22.12,18.72
375692,7/21/16,0.866667,18.72,22.12,17.60,21.86,21.86,18.42,22.01,18.72
375693,7/22/16,0.866667,18.72,22.01,16.79,21.86,21.86,17.60,21.89,18.72


In [8]:
train_df_form.isna().sum()

startdate                      0
lon                            0
nmme0-tmp2m-34w__ccsm30    15934
nmme-tmp2m-56w__ccsm3      10280
nmme-prate-34w__ccsm3       8738
nmme0-prate-56w__ccsm30    15934
nmme0-prate-34w__ccsm30    15934
nmme-prate-56w__ccsm3      10280
nmme-tmp2m-34w__ccsm3       8738
ccsm30                     15934
dtype: int64

The question is, "What can we do with these nulls?".

The one simple approach is to replace nulls with their correct value. In this case, having some prior knowledge, I can tell you that these nulls are actually supposed to be the value for respective columns depending on their start date. I know this because only certain zones were sampled in this data and based on their longitude values and associated latitudes values. 

In many cases, you will not really know how to fill in null values so you can remove any rows with nulls, impute their value with an average (or some other estimator), or take other more sophisticated approaches.

Let's simply select all the null values rows to view it and replace them with the previous knowledge of its corresponding lat and long values.



In [9]:
# Select all rows with NaN under the entire train_df
train_rw_nans = train_df[train_df.isna().any(axis=1)]
train_rw_nans.head(50)

,index,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,...,wind-vwnd-925-2010-11,wind-vwnd-925-2010-12,wind-vwnd-925-2010-13,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20
356,356,0.0,0.833333,8/23/15,299.07,30.07,32.31,NaN,31.28,29.69,...,-29.02,-12.80,54.28,-18.25,7.51,57.60,-71.31,8.55,-11.71,3.98
357,357,0.0,0.833333,8/24/15,305.88,30.07,32.31,NaN,31.28,29.69,...,-28.10,-14.45,59.29,-20.98,1.43,64.69,-58.28,8.69,-11.43,5.50
358,358,0.0,0.833333,8/25/15,312.20,30.07,32.31,NaN,31.28,29.69,...,-24.23,-24.15,64.92,-24.31,-5.21,67.95,-43.36,11.13,-13.36,9.91
359,359,0.0,0.833333,8/26/15,319.21,30.07,32.31,NaN,31.28,29.69,...,-10.38,-36.97,64.12,-25.82,-9.52,65.41,-27.02,12.14,-14.03,13.06
360,360,0.0,0.833333,8/27/15,324.65,30.07,32.31,NaN,31.28,29.69,...,9.28,-39.72,58.01,-21.25,-8.79,65.20,-14.07,10.10,-12.56,7.55
361,361,0.0,0.833333,8/28/15,320.91,30.07,32.31,NaN,31.28,29.69,...,26.26,-27.57,49.90,-10.91,-13.70,65.36,-5.23,9.89,-12.78,6.09
362,362,0.0,0.833333,8/29/15,302.89,30.07,32.31,NaN,31.28,29.69,...,34.56,-13.52,43.66,0.39,-23.05,65.31,3.23,15.93,-10.80,6.22
363,363,0.0,0.833333,8/30/15,279.86,30.07,32.31,NaN,31.28,29.69,...,35.69,1.11,36.72,6.91,-23.56,62.87,10.30,25.90,-10.88,2.93
364,364,0.0,0.833333,8/31/15,268.37,30.07,32.31,NaN,31.28,29.69,...,28.12,15.48,27.34,16.12,-20.38,59.71,16.82,35.83,-22.08,4.85
365,365,0.0,0.833333,9/1/15,274.54,30.07,32.31,NaN,31.28,29.69,...,17.09,25.45,18.81,23.53,-19.84,55.51,19.55,43.56,-30.86,4.93


Having look at the table above that displays the rows with NaNs, we will go on to replace each affected columns with a prior knowledge of existing value or corresponding value related to its longitude.

In [10]:
# making a smaller copy of the data (one with less columns with NANs excluding startdate and longitude) 
train_df_form_1=train_rw_nans[['startdate','lon','nmme0-tmp2m-34w__ccsm30', 'nmme-tmp2m-56w__ccsm3', 'nmme-prate-34w__ccsm3', 'nmme0-prate-56w__ccsm30', 'nmme0-prate-34w__ccsm30', 'nmme-prate-56w__ccsm3', 'nmme-tmp2m-34w__ccsm3', 'ccsm30']].copy()
train_df_form_1.head(50)

,startdate,lon,nmme0-tmp2m-34w__ccsm30,nmme-tmp2m-56w__ccsm3,nmme-prate-34w__ccsm3,nmme0-prate-56w__ccsm30,nmme0-prate-34w__ccsm30,nmme-prate-56w__ccsm3,nmme-tmp2m-34w__ccsm3,ccsm30
356,8/23/15,0.833333,NaN,28.88,0.64,19.37,NaN,8.31,0.64,28.88
357,8/24/15,0.833333,NaN,28.67,0.57,19.37,NaN,9.53,0.57,28.88
358,8/25/15,0.833333,NaN,28.46,0.50,19.37,NaN,10.74,0.50,28.88
359,8/26/15,0.833333,NaN,28.26,0.43,19.37,NaN,11.96,0.43,28.88
360,8/27/15,0.833333,NaN,28.05,0.36,19.37,NaN,13.18,0.36,28.88
361,8/28/15,0.833333,NaN,27.84,0.29,19.37,NaN,14.40,0.29,28.88
362,8/29/15,0.833333,NaN,27.64,0.21,19.37,NaN,15.61,0.21,28.88
363,8/30/15,0.833333,NaN,27.22,0.14,19.37,NaN,18.05,0.14,28.88
364,8/31/15,0.833333,NaN,27.02,0.07,19.37,NaN,19.27,0.07,28.88
365,9/1/15,0.833333,NaN,26.81,NaN,19.37,NaN,20.48,NaN,28.88


In [11]:
#Replacing the NaNs in the affected columns with its corresponding choice of values
#based on the prior knowledge or long.

#train_df['nmme0-tmp2m-34w__ccsm30']=train_df.nmme0-tmp2m-34w__ccsm30.fillna(41)

In [12]:
test_df.isna().sum()

index                                 0
lat                                   0
lon                                   0
startdate                             0
contest-pevpr-sfc-gauss-14d__pevpr    0
                                     ..
wind-vwnd-925-2010-16                 0
wind-vwnd-925-2010-17                 0
wind-vwnd-925-2010-18                 0
wind-vwnd-925-2010-19                 0
wind-vwnd-925-2010-20                 0
Length: 245, dtype: int64

Format the 'startdate' colummn to 'datetime' 

In [13]:
# convert start_date column to a datetime type
train_df['startdate'] = pd.to_datetime(train_df['startdate'])
test_df['startdate'] = pd.to_datetime(test_df['startdate'])


In [14]:
# group start_date by month and check averages of the other columns
train_df.groupby(pd.Grouper(key='startdate',freq='M')).mean()
test_df.groupby(pd.Grouper(key='startdate',freq='M')).mean()

,index,lat,lon,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,nmme0-tmp2m-34w__gfdlflora0,...,wind-vwnd-925-2010-11,wind-vwnd-925-2010-12,wind-vwnd-925-2010-13,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20
startdate,,,,,,,,,,,,,,,,,,,,,
2022-11-30,391395.0,0.592766,0.517964,354.747025,23.089105,24.886314,22.268392,23.044110,20.879891,23.563319,...,1.717000,-54.984333,6.681333,45.268667,-35.728333,12.138333,4.089667,-10.018333,-1.957000,26.068000
2022-12-31,391425.5,0.592766,0.517964,245.532173,15.757553,17.624336,16.239224,17.330387,15.727886,17.332059,...,41.689355,-14.438387,43.683871,28.827419,-7.318065,8.813548,-28.902581,25.854194,-20.691935,16.260323
